# Workflow

### Importación de librerías

In [ ]:
import workflow as wf
import shap

#### Recarga del script en el noteook si se realizan cambios

In [ ]:
import importlib
importlib.reload(wf)

### Carga y preprocesamiento de los datos

#### Path a los datos y variable obetivo

In [ ]:
path = "REGISTRO  FERNANDO PARA IA.xls"
target = "EXITUS30D"

#### Carga de los datos y estandarización de los mismos:
- missing values: [' ', 'NaN', 'na', 'Na', '-', '--', 'n/a']
- bools: ['Sí', 'sí', 'SI', 'Si'] -> 'si', lo mismo para 'No'
- lower

In [ ]:
datos_raw = wf.load_and_clean(path)

#### Eliminación de aquellas columnas no relevantes para el estudio

In [ ]:
datos_raw = wf.drop_initials_columns(datos_raw)

In [ ]:
datos_raw.shape

In [ ]:
datos_raw

#### Codificación de los datos, tres técnicas disponibles:
- Para utilizar técnicas de muestreo:
  - ordinal encoding - `wf.ordinal_encoding()`
- Para entrenar directamente los modelos neuronales
  - binary enconding - `wf.binary_encoding()`
  - one hot encoding - `wf.one_hot_encoding()`

In [ ]:
data_encoded = wf.ordinal_encoding(datos_raw)

#### Estudio de correlación de los datos:
- Test de Kruskal-Wallis para conjuntos de datos numéricos y categóricos - `wf.kruskal_wallis_test()` (Si Plot=True, mostramos en el output el p-valor de cada variable)
- Test Chi-cuadrado para conjuntos de datos categóricos - `wf.chi_squared_test()`

In [ ]:
datos_kruskal = wf.kruskal_wallis_test(data_encoded, target, plot=False)

In [ ]:
datos_kruskal.shape

#### Test para la selección de porcentae máximo de missing values y técnica de imputación de los mismos:
- Porcentajes de missing values a probar: 5%, 10%, 15%, 20%, 40% y 60%
- Técnicas de imputación aplicadas:
  - Media
  - Moda
  - K Nearest Neighbors
  - Random Forest
- Cálculo de MSE para validar cada una de las combinaciones posibles

In [ ]:
wf.imputation_tests(datos_kruskal, target)

#### Eliminación de missing values en función del porcentaje máximo permitido seleccionado

In [ ]:
data_dropped = wf.drop_missing_values_columns(datos_kruskal, 5)

In [ ]:
data_dropped.shape

#### Imputación de los missing values restantes aplicando la técnica con mejores resultados del test
- Opciones:
    - `wf.mode_imputation()`
    - `wf.mean_imputation()`
    - `wf.knn_imputation()`
    - `wf.random_forest_imputation()`

In [ ]:
datos_mode = wf.mode_imputation(data_dropped, target)

#### Guardado de los datos limpios, codificados y sin ningún missing value

In [ ]:
datos_mode.to_excel('datasets_generados/datos_codificados_y_rellenados.xlsx')

#### Normalización o Estandarización de los datos a una misma escala
- Opciones posibles:
    - MinMax normalization (`wf.min_max_normalization()`)
    - Standard Scaler (`wf.standar_scaler()`)

In [ ]:
data_normalize = wf.min_max_normalization(datos_mode)

#### Guardado de los datos normalizados o estandarizados (cambiar nombre)

In [ ]:
data_normalize.to_excel('datasets_generados/datos_normalizados.xlsx')

In [ ]:
data_normalize

#### Creación de los conjuntos de test con datos originales, antes de aplicar el muestreo

In [ ]:
x_test_original = data_normalize[data_normalize[target]==1]
x_test_original = x_test_original.append(data_normalize[data_normalize[target]==0])
x_test_original = x_test_original.iloc[0:58]
y_test_original = x_test_original[target]
x_test_original = x_test_original.drop(columns=[target])

### Original data

#### Preparación de los datos y creación de conjuntos de train y test de cara al entrenamiento de los modelos

In [ ]:
data_normalize_dropped = data_normalize.drop(columns = [target])

In [ ]:
x_train, x_test, y_train, y_test, network_output = wf.prep_datos_red(target , data_normalize, data_normalize_dropped)

#### Grid Search para la búsqueda de los mejores parámetros:
- Para establecer o modificar estos ir a `def grid_search_mlp()` en `workflow.py`

In [ ]:
best_params = wf.grid_search_mlp(x_train, y_train, x_test, y_test)

#### Parámetros seleccionados manualmente para saltar el proceso del Grid Search en reiteradas ocasiones,  saltar a la línea siguiente al ejecutar para seleccionar los mejores

In [ ]:
best_params = {'batch_size': 16, 'dropout_rate': 0.2, 'epochs': 50, 'neurons': 12}

In [ ]:
best_params

##### Compatibilidad de los modelos tanto con TensorFlow 1.0 como con 2.0

In [ ]:
wf.tf.compat.v1.disable_v2_behavior()

##### Limpiamos todos aquellos modelos que se han quedado guardados en memoria

In [ ]:
wf.tf.keras.backend.clear_session()

#### Entrenamiento de múltiples MLPs y guardado de los mismos
- n = 10 es el número de modelos a entrenar
- patience = 5 es el parámetro para `wf.tf.keras.callbacks.EarlyStopping`

In [ ]:
wf.train_multiple_models(x_train, y_train, x_test_original, y_test_original, 10, best_params, 'models/normal/model_', 5)

#### Carga de los modelos desde el path donde están almacenados y realización de la media entre las capas de los modelos

In [ ]:
best_model = wf.load_and_ensemble_best_model(10, 'models/normal/model_', plot=True)

#### Entrenamiento del mejor modelo sobre el conjunto de train generado anteriormente

In [ ]:
best_model_train = wf.train_model(best_model, best_params, x_train, y_train, patience=5)

#### Predicción sobre el conjunto de datos originales:
- Obtención de las métricas de:
    - Accuracy
    - Precision
    - Recall
    - F1 Score
- Matriz de Confusión(TP, TN, FP, FN)

In [ ]:
predicts = wf.predict_model_and_report(best_model, x_test_original, y_test_original, ['exitus', 'no exitus'])

### SMOTE + Tomed Links data

#### Aplicación de la técnicas de muestro para desbalancear los datos

In [ ]:
data_balanced = wf.smote_tomed_link(data_normalize, target)

In [ ]:
data_balanced[target].value_counts()

In [ ]:
data_balanced.to_excel('datasets_generados/datos_balanceados_smote_tomed_links.xlsx')

#### Preparación de los datos y creación de conjuntos de train y test de cara al entrenamiento de los modelos

In [ ]:
data_balanced_dropped = data_balanced.drop(columns = [target])

In [ ]:
x_train, x_test, y_train, y_test, network_output = wf.prep_datos_red(target , data_balanced, data_balanced_dropped)

#### Grid Search para la búsqueda de los mejores parámetros:
- Para establecer o modificar estos ir a `def grid_search_mlp()` en `workflow.py`

In [ ]:
best_params = wf.grid_search_mlp(x_train, y_train, x_test_original, y_test_original)

#### Parámetros seleccionados manualmente para saltar el proceso del Grid Search en reiteradas ocasiones,  saltar a la línea siguiente al ejecutar para seleccionar los mejores

In [ ]:
best_params = {'batch_size': 16, 'dropout_rate': 0.25, 'epochs': 50, 'neurons': 8}

In [ ]:
best_params

##### Compatibilidad de los modelos tanto con TensorFlow 1.0 como con 2.0

In [ ]:
wf.tf.compat.v1.disable_v2_behavior()

##### Limpiamos todos aquellos modelos que se han quedado guardados en memoria

In [ ]:
wf.tf.keras.backend.clear_session()

#### Entrenamiento de múltiples MLPs y guardado de los mismos
- n = 10 es el número de modelos a entrenar
- patience = 5 es el parámetro para `wf.tf.keras.callbacks.EarlyStopping`

In [ ]:
wf.train_multiple_models(x_train, y_train, x_test_original, y_test_original, 10, best_params, 'models/normal/model_', 3)

#### Carga de los modelos desde el path donde están almacenados y realización de la media entre las capas de los modelos

In [ ]:
best_model = wf.load_and_ensemble_best_model(10, 'models/normal/model_', plot=True)

#### Entrenamiento del mejor modelo sobre el conjunto de train generado anteriormente

In [ ]:
best_model_train = wf.train_model(best_model, best_params, x_train, y_train, patience=2)

#### Predicción sobre el conjunto de datos originales:
- Obtención de las métricas de:
    - Accuracy
    - Precision
    - Recall
    - F1 Score
- Matriz de Confusión(TP, TN, FP, FN)

In [ ]:
predicts = wf.predict_model_and_report(best_model, x_test_original, y_test_original, ['exitus', 'no exitus'])

### SMOTE + ENN data

#### Aplicación de la técnicas de muestro para desbalancear los datos

In [ ]:
data_balanced = wf.smote_edited_nearest_neighbor(data_normalize, target)

In [ ]:
data_balanced.to_excel('datasets_generados/datos_balanceados_smote_enn.xlsx')

In [ ]:
data_balanced[target].value_counts()

#### Preparación de los datos y creación de conjuntos de train y test de cara al entrenamiento de los modelos

In [ ]:
data_balanced_dropped = data_balanced.drop(columns = [target])

In [ ]:
x_train, x_test, y_train, y_test, network_output = wf.prep_datos_red(target , data_balanced, data_balanced_dropped)

#### Grid Search para la búsqueda de los mejores parámetros:
- Para establecer o modificar estos ir a `def grid_search_mlp()` en `workflow.py`

In [ ]:
best_params = wf.grid_search_mlp(x_train, y_train, x_test, y_test)

#### Parámetros seleccionados manualmente para saltar el proceso del Grid Search en reiteradas ocasiones,  saltar a la línea siguiente al ejecutar para seleccionar los mejores

In [ ]:
best_params = {'batch_size': 16, 'dropout_rate': 0.3, 'epochs': 50, 'neurons': 12}

In [ ]:
best_params

##### Compatibilidad de los modelos tanto con TensorFlow 1.0 como con 2.0

In [ ]:
wf.tf.compat.v1.disable_v2_behavior()

##### Limpiamos todos aquellos modelos que se han quedado guardados en memoria

In [ ]:
wf.tf.keras.backend.clear_session()

#### Entrenamiento de múltiples MLPs y guardado de los mismos
- n = 10 es el número de modelos a entrenar
- patience = 5 es el parámetro para `wf.tf.keras.callbacks.EarlyStopping`

In [ ]:
wf.train_multiple_models(x_train, y_train, x_test_original, y_test_original, 10, best_params, 'models/normal/model_', 5)

#### Carga de los modelos desde el path donde están almacenados y realización de la media entre las capas de los modelos

In [ ]:
best_model = wf.load_and_ensemble_best_model(10, 'models/normal/model_', plot=True)

#### Entrenamiento del mejor modelo sobre el conjunto de train generado anteriormente

In [ ]:
best_model_train = wf.train_model(best_model, best_params, x_train, y_train, patience=5)

#### Predicción sobre el conjunto de datos originales:
- Obtención de las métricas de:
    - Accuracy
    - Precision
    - Recall
    - F1 Score
- Matriz de Confusión(TP, TN, FP, FN)

In [ ]:
predicts = wf.predict_model_and_report(best_model_train, x_test_original, y_test_original, ['exitus', 'no exitus'])

### Shap Deep Explainer

In [ ]:
deep_explainer, deep_values, x_train_deep_df = wf.shap_deep_explainer(data_balanced_dropped, x_train, x_test_original, best_model_train)

#### Force Plot que representa los valores de Shappley para cada atributo a lo largo de todas las predicciones

In [ ]:
shap.force_plot(deep_explainer.expected_value, deep_values[0], x_train_deep_df, link="logit")

#### Plot con el sumatorio de los valores de Shappley para cada atributo, obteniendo como resultado las características más importantes en las decisiones de los modelos

In [ ]:
wf.shap_summary_plot(deep_values, x_train_deep_df)

##### Creación de los conjuntos de datos(train) a partir de este subconjunto de características

In [ ]:
deep_features, deep_features_drop, features = wf.features_df(data_balanced, deep_values, 8, target)

#### Conjuntos de test tras la reducción de las características o variables menos influyentes

In [ ]:
x_test_original_df = x_test_original[features]
y_test_original_df = y_test_original

#### Preparación de los datos y creación de conjuntos de train y test de cara al entrenamiento de los modelos

In [ ]:
x_train_df, x_test_df, y_train_df, y_test_df, network_output_df = wf.prep_datos_red(target, deep_features, deep_features_drop)

##### Limpiamos todos aquellos modelos que se han quedado guardados en memoria

In [ ]:
tf.keras.backend.clear_session()

#### Entrenamiento de múltiples MLPs y guardado de los mismos
- n = 10 es el número de modelos a entrenar
- patience = 5 es el parámetro para `wf.tf.keras.callbacks.EarlyStopping`

In [ ]:
wf.train_multiple_models(x_train_df, y_train_df, x_test_original_df, y_test_original_df, 10, best_params, 'models/features/model_', 5)

#### Carga de los modelos desde el path donde están almacenados y realización de la media entre las capas de los modelos

In [ ]:
best_model = wf.load_and_ensemble_best_model(10, 'models/features/model_', plot=True)

#### Entrenamiento del mejor modelo sobre el conjunto de train generado anteriormente

In [ ]:
best_model_train_df = wf.train_model(best_model, best_params, x_train_df, y_train_df, patience=10)

#### Predicción sobre el conjunto de datos originales:
- Obtención de las métricas de:
    - Accuracy
    - Precision
    - Recall
    - F1 Score
- Matriz de Confusión(TP, TN, FP, FN)

In [ ]:
predicts_df = wf.predict_model_and_report(best_model_train_df, x_test_original_df, y_test_original_df, ['exitus', 'no exitus'])

#### Guardamos las predicciones realizadas por el modelo para usarlas posteriormente en los Árboles de Decisión

In [ ]:
predicts_deep = wf.predict_model_and_report(best_model_train_df, x_train_df, y_train_df, ['exitus', 'no exitus'])

### BASELINE NO NEURONALES

#### Sobre los datos tras aplicar la reducción de características:
- K Nearest Neigghbors
- Support Vector Machine
- Logistic Regression

In [ ]:
wf.knn_classifier(x_train_df, y_train_df, x_test_original_df, y_test_original_df)

In [ ]:
wf.svm_classifier(x_train_df, y_train_df, x_test_original_df, y_test_original_df)

In [ ]:
wf.lr_classifier(x_train_df, y_train_df, x_test_original_df, y_test_original_df)

### Árboles de Decisión

#### CART: Implementación tradicional, entrenando el modelo jerárquico directamente sobre los datos

##### Extraemos lo siguiente:
- Obtención de las métricas de:
    - Accuracy
    - Precision
    - Recall
    - F1 Score
- Matriz de Confusión(TP, TN, FP, FN)
- Conjunto de reglas

In [ ]:
cart_tree, cart_tree_rules, features = wf.cart_decision_tree(deep_features_drop, x_train_df, predicts_deep, x_test_original_df, y_test_original_df, 'entropy')

##### Generación del plot con el árbol de decisión resultante

In [ ]:
dt_graph = wf.plot_tree(cart_tree, features)
Image(dt_graph.create_png())

#### TREPAN: Modelo jerárquico que utilizado el modelo neuronal y sus predicciones como complemento en el proceso de entrenamiento y predicción

##### Creación del modelo jerárquico con los datos de entrenamiento y creación del modelo complementario basado en nuestro modelo neuronal y el conjunto de test que usará para ayudarse en el proceso de entrenamiento

In [ ]:
interpreter = Interpretation(training_data=x_train_df, feature_names=features)
im_model = InMemoryModel(best_model_train_df.predict, examples=x_test_original_df, feature_names=features, unique_values=[0, 1])

##### Instanciación del modelo completo al cual podemos establecer una profundidad máxima

In [ ]:
surrogate_explainer = TreeSurrogate(oracle=im_model, seed=42, max_depth=x_train_df.shape[1]-1)

##### Entrenamiento del modelo usando el modelo neuronal como ayuda. Podemos establecer se queremos que haga pre proda o post poda al explorar nodos

In [ ]:
surrogate_explainer.fit(x_train_df, predicts_deep, use_oracle=True, prune='pre')

#### Predicción sobre el conjunto de datos originales:
- Obtención de las métricas de:
    - Accuracy
    - Precision
    - Recall
    - F1 Score
- Matriz de Confusión(TP, TN, FP, FN)

In [ ]:
predicts = wf.predict_model_and_report(surrogate_explainer, x_test_original_df, y_test_original_df, ['exitus', 'no exitus'])

In [ ]:
graph = Source(surrogate_explainer.plot_global_decisions(colors=['coral', 'darkturquoise'], 
                                          file_name='test_tree_pre.png').to_string())
svg_data = graph.pipe(format='svg')
with open('dtree_structure.svg','wb') as f:
    f.write(svg_data)
SVG(svg_data)